In [202]:
import os
import pandas as pd
import numpy as np


In [218]:
import re

PATH = './data/raw/'
program = 'PERM'

list_files = os.listdir(os.path.join(PATH, program))

years = [str(year) for year in range(2008, 2025)]

data = {}

for year in years:
    year_files = [file for file in list_files if year in file]
    if not year_files:
        continue

    year_data = None
    quarter_files = {}

    for file in year_files:
        quarter = re.findall(r'Q\d', file)
        if quarter:
            quarter_files[quarter[0]] = file
        else:
            year_data = pd.read_csv(f'{os.path.join(PATH, program)}/{file}', low_memory=False)

    if year_data is not None:
        data[year] = year_data
    elif 'Q4' in quarter_files:
        data[year] = pd.read_csv(f'{os.path.join(PATH, program)}/{quarter_files["Q4"]}', low_memory=False)


In [219]:
# Modify columns to be more similar
for year in years:
    if year in data:
        # Remove leading and trailing spaces, convert to uppercase, and replace spaces with underscores
        data[year].columns = data[year].columns.str.upper().str.replace(' ', '_')
        # Replace EMP with EMPLOYER
        data[year].rename(columns={'EMP': 'EMPLOYER'}, inplace=True)
        # Replace CASE_NO with CASE_NUMBER
        data[year].rename(columns={'CASE_NO': 'CASE_NUMBER'}, inplace=True)
        # Replace WAGE_OFFER with WAGE_OFFERED in all columns
        data[year].rename(columns=lambda x: re.sub(r'WAGE_OFFER_', 'WAGE_OFFERED_', x), inplace=True)
        # Solve spelling issues:
        data[year].rename(columns={
                            'COUNTRY_OF_CITZENSHIP': 'COUNTRY_OF_CITIZENSHIP'
                            }, inplace=True)
        # Remove 2007_ from 2007_NAICS_US_CODE and 2007_NAICS_US_TITLE
        data[year].rename(columns={
            '2007_NAICS_US_CODE': 'NAICS_CODE', 
            '2007_NAICS_US_TITLE': 'NAICS_TITLE',
            'NAICS_US_CODE' : 'NAICS_CODE',
            'NAICS_US_TITLE' : 'NAICS_TITLE',
            'WAGE_OFFERED_FROM_9089' : 'WAGE_OFFERED_FROM',
            'WAGE_OFFERED_TO_9089' : 'WAGE_OFFERED_TO',
            'WAGE_OFFERED_UNIT_OF_PAY_9089' : 'WAGE_OFFERED_UNIT_OF_PAY'
            }, inplace=True)
        # Add WAGE_OFFERED_UNIT_OF_PAY_9089 column if column does not exist
        if 'WAGE_OFFERED_UNIT_OF_PAY' not in data[year].columns:
            data[year]['WAGE_OFFERED_UNIT_OF_PAY'] = np.nan
        # Rename EMPLOYER_STATE_PROVINCE to EMPLOYER_STATE
        data[year].rename(columns={'EMPLOYER_STATE_PROVINCE': 'EMPLOYER_STATE'}, inplace=True)
        # Replace JOB_INFO_WORK_ with WORKSITE_
        data[year].rename(columns=lambda x: re.sub(r'JOB_INFO_WORK_', 'WORKSITE_', x), inplace=True)
        # Drop any columns that starts with "PW_" (this si the prevailing wage we'll get it from somewhere else)
        data[year].drop(columns=data[year].filter(regex='^PW_').columns, inplace=True)
        # ! NOTE dropping the following columns because they are not present in all years
        cols_to_drop = ['APPLICATION_TYPE', 'US_ECONOMIC_SECTOR', "EMPLOYER_ADDRESS_2", "NAICS_TITLE"]
        data[year].drop(columns=cols_to_drop, errors='ignore', inplace=True)
        
        

common_columns = set(data[years[0]].columns)

print(f"Year = {years[0]} number of columns = {len(common_columns)}")
for year in years[1:]:
    print(f"Year = {year}, number of common columns = {len(common_columns)}")
    if year in data:
        common_columns.intersection_update(data[year].columns)

common_columns = list(common_columns)
print(common_columns)

# Create a long dataframe with all the data from all years filtered by the common columns
data_long = pd.DataFrame()
for year in years:
    if year in data:
        data_long = pd.concat([data_long, data[year][common_columns]], ignore_index=True)

data_long

Year = 2008 number of columns = 16
Year = 2009, number of common columns = 16
Year = 2010, number of common columns = 16
Year = 2011, number of common columns = 16
Year = 2012, number of common columns = 16
Year = 2013, number of common columns = 16
Year = 2014, number of common columns = 16
Year = 2015, number of common columns = 16
Year = 2016, number of common columns = 16
Year = 2017, number of common columns = 16
Year = 2018, number of common columns = 16
Year = 2019, number of common columns = 16
Year = 2020, number of common columns = 16
Year = 2021, number of common columns = 16
Year = 2022, number of common columns = 16
Year = 2023, number of common columns = 16
Year = 2024, number of common columns = 16
['EMPLOYER_CITY', 'CASE_STATUS', 'WORKSITE_CITY', 'WAGE_OFFERED_UNIT_OF_PAY', 'EMPLOYER_NAME', 'CASE_NUMBER', 'DECISION_DATE', 'NAICS_CODE', 'CLASS_OF_ADMISSION', 'EMPLOYER_ADDRESS_1', 'WAGE_OFFERED_FROM', 'WORKSITE_STATE', 'EMPLOYER_POSTAL_CODE', 'COUNTRY_OF_CITIZENSHIP', 'EM

,EMPLOYER_CITY,CASE_STATUS,WORKSITE_CITY,WAGE_OFFERED_UNIT_OF_PAY,EMPLOYER_NAME,CASE_NUMBER,DECISION_DATE,NAICS_CODE,CLASS_OF_ADMISSION,EMPLOYER_ADDRESS_1,WAGE_OFFERED_FROM,WORKSITE_STATE,EMPLOYER_POSTAL_CODE,COUNTRY_OF_CITIZENSHIP,EMPLOYER_STATE,WAGE_OFFERED_TO
0,WASHINGTON,DENIED,WASHINGTON,HR,DC GRILL INC T/A DC CAFE,A-08271-91262,29-Sep-08,NaN,A1/A2,2035 P STREET NW,10.23,DC,NaN,MAURITANIA,DC,NaN
1,LOS ANGELES,DENIED,LOS ANGELES,YR,NAG INC DBA ENGINEERING SYSTEMS,C-07327-98303,29-Nov-07,NaN,A1/A2,355 SOUTH GRAND AVENUE,54000.0,CA,NaN,PAKISTAN,CA,NaN
2,BALTIMORE,CERTIFIED,BALTIMORE,HR,"UNION ENTERPRISES, INC.",A-08029-18103,10-Jul-08,NaN,A-3,7821 WISE AVENUE,9.8,MD,NaN,PHILIPPINES,MD,NaN
3,CHEVERLY,DENIED,CHEVERLY,HR,"CIVIL CONSTRUCTION, LLC.",A-07262-76878,15-Oct-07,NaN,A-3,2413 SCHUSTER DR.,11.42,MD,NaN,PORTUGAL,MD,NaN
4,TEANECK,DENIED,TEANECK,HR,AMSERA GENERAL BEAUTY MERCHANDISE,A-08273-91603,30-Sep-08,NaN,B-1,1470 GAYLORD TERRACE,6.0,NJ,NaN,SIERRA LEONE,NJ,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2319446,BLOOMINGTON,Denied,Bloomington,Year,VALLEY ALDER FAMILY DENTISTRY,A-21175-06722,09-30-24,621210,H-1B,17644 VALLEY BLVD,116293.0,CALIFORNIA,92316,INDIA,CALIFORNIA,NaN
2319447,MINNEAPOLIS,Denied,MINNEAPOLIS,Hour,MARHABA RESTAURANT,A-22210-97191,09-30-24,722511,NaN,2801 NICOLLET AVENUE,18.0,MINNESOTA,55408,EGYPT,MINNESOTA,NaN
2319448,NASHVILLE,Denied,New York,Year,UBS SECURITIES LLC,A-21243-38341,09-30-24,523110,H-1B,315 DEADERICK STREET,140000.0,NEW YORK,37238,IRELAND,TENNESSEE,NaN
2319449,Cocoa,Certified,Cocoa,Year,Luke Gell Pools LLC,A-22187-83701,09-30-24,561790,E-2,4350 N. U.S. 1,34466.0,FLORIDA,32927,UNITED KINGDOM,FLORIDA,NaN


In [253]:
import re

PATH = './data/raw/'
program = 'H-1B'

list_files = os.listdir(os.path.join(PATH, program))

years = [str(year) for year in range(2008, 2025)]

data = {}

for year in years:
    year_files = [file for file in list_files if year in file]
    if not year_files:
        continue

    year_data = None
    quarter_files = {}

    for file in year_files:
        quarter = re.findall(r'Q\d', file)
        if quarter:
            quarter_files[quarter[0]] = file
        else:
            year_data = pd.read_csv(f'{os.path.join(PATH, program, file)}', low_memory=False)

    if year_data is not None:
        data[year] = year_data
    elif 'Q4' in quarter_files:
        data[year] = pd.read_csv(f'{os.path.join(PATH, program, quarter_files["Q4"])}', low_memory=False)



In [257]:
# Modify columns to be more similar
for year in years:
    if year in data:
        # Remove leading and trailing spaces, convert to uppercase, and replace spaces with underscores
        data[year].columns = data[year].columns.str.upper().str.replace(' ', '_')
        # Remove LCA_CASE_ from all columns
        data[year].columns = data[year].columns.str.replace('LCA_CASE_', '')
        # Replace EMP with EMPLOYER
        data[year].rename(columns={'EMP': 'EMPLOYER'}, inplace=True)
        # Replace CASE_NO with CASE_NUMBER
        data[year].rename(columns={'CASE_NO': 'CASE_NUMBER'}, inplace=True)
        # Replace WAGE_OFFER with WAGE_OFFERED in all columns
        data[year].rename(columns=lambda x: re.sub(r'WAGE_OFFER_', 'WAGE_OFFERED_', x), inplace=True)
        # Solve spelling issues:
        data[year].rename(columns={
                            'COUNTRY_OF_CITZENSHIP': 'COUNTRY_OF_CITIZENSHIP'
                            }, inplace=True)
        # Remove 2007_ from 2007_NAICS_US_CODE and 2007_NAICS_US_TITLE
        data[year].rename(columns={
            '2007_NAICS_US_CODE': 'NAICS_CODE', 
            '2007_NAICS_US_TITLE': 'NAICS_TITLE',
            'NAICS_US_CODE' : 'NAICS_CODE',
            'NAICS_US_TITLE' : 'NAICS_TITLE',
            'WAGE_OFFERED_FROM_9089' : 'WAGE_OFFERED_FROM',
            'WAGE_OFFERED_TO_9089' : 'WAGE_OFFERED_TO',
            'WAGE_OFFERED_UNIT_OF_PAY_9089' : 'WAGE_OFFERED_UNIT_OF_PAY'
            }, inplace=True)
        # Add WAGE_OFFERED_UNIT_OF_PAY_9089 column if column does not exist
        if 'WAGE_OFFERED_UNIT_OF_PAY' not in data[year].columns:
            data[year]['WAGE_OFFERED_UNIT_OF_PAY'] = np.nan
        # Rename EMPLOYER realated columns
        data[year].rename(columns={
            'EMPLOYER_STATE_PROVINCE': 'EMPLOYER_STATE',
            'EMPLOYER_ADDRESS1': 'EMPLOYER_ADDRESS',
            'WORKLOC1_STATE': 'WORK_LOCATION_STATE1',
            'WORKLOC2_STATE': 'WORK_LOCATION_STATE2',
            'WORKLOC1_CITY' : 'WORK_LOCATION_CITY1' ,
            'WORKLOC2_CITY' : 'WORK_LOCATION_CITY2'
            }, inplace=True)
        # Replace JOB_INFO_WORK_ with WORKSITE_
        data[year].rename(columns=lambda x: re.sub(r'JOB_INFO_WORK_', 'WORKSITE_', x), inplace=True)
        # Drop any columns that starts with "PW_" (this si the prevailing wage we'll get it from somewhere else)
        data[year].drop(columns=data[year].filter(regex='^PW_').columns, inplace=True)
        # ! NOTE dropping the following columns because they are not present in all years
        cols_to_drop = ['APPLICATION_TYPE', 'US_ECONOMIC_SECTOR', "EMPLOYER_ADDRESS_2",
                        "EMPLOYER_ADDRESS2", "NAICS_TITLE"]
        data[year].drop(columns=cols_to_drop, errors='ignore', inplace=True)

In [258]:
years = list(data.keys())
years.sort()
common_columns = set(data[years[0]].columns)

print(f"Year = {years[0]} number of columns = {len(common_columns)}")
for year in years[1:]:
    print(f"Year = {year}, number of common columns = {len(common_columns)}")
    if year in data:
        common_columns.intersection_update(data[year].columns)

common_columns = list(common_columns)
print(common_columns)

# Create a long dataframe with all the data from all years filtered by the common columns
data_long = pd.DataFrame()
for year in years:
    if year in data:
        data_long = pd.concat([data_long, data[year][common_columns]], ignore_index=True)

data_long

Year = 2010 number of columns = 25
Year = 2013, number of common columns = 25
Year = 2014, number of common columns = 25
Year = 2015, number of common columns = 25
Year = 2016, number of common columns = 13
Year = 2017, number of common columns = 13
Year = 2018, number of common columns = 13
Year = 2019, number of common columns = 13
['EMPLOYER_CITY', 'EMPLOYER_POSTAL_CODE', 'JOB_TITLE', 'WAGE_OFFERED_UNIT_OF_PAY', 'EMPLOYER_NAME', 'DECISION_DATE', 'SOC_CODE', 'EMPLOYER_ADDRESS', 'EMPLOYER_STATE']


,EMPLOYER_CITY,EMPLOYER_POSTAL_CODE,JOB_TITLE,WAGE_OFFERED_UNIT_OF_PAY,EMPLOYER_NAME,DECISION_DATE,SOC_CODE,EMPLOYER_ADDRESS,EMPLOYER_STATE
0,COLUMBIA,21045.0,NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS,NaN,XPEDITE TECHNOLOGIES INC,10-01-09,15-1071.00,8830 STANFORD BLVD,MD
1,NORTH BRUNSWICK,8902.0,PROGRAMMER ANALYST,NaN,CYBER RESOURCE GROUP INC,10-01-09,15-1021.00,208 NORTH CENTER DRIVE,NJ
2,BOISE,83716.0,SALES REPORTING ANALYST,NaN,"MICRON TECHNOLOGY, INC.",10-01-09,15-1031.00,"8000 S. FEDERAL WAY, MAIL STOP 1-507",ID
3,BETHESDA,20892.0,RESEARCH FELLOW,NaN,"NATIONAL INSTITUTES OF HEALTH, HHS",10-01-09,19-1021.00,DIVISION OF INTERNATIONAL SERVICES/ORS,MD
4,PISCATAWAY,8854.0,PROGRAMMER/ANALYST,NaN,"MARLABS, INC",10-01-09,15-1021.00,1 CORPORATE PLACE SOUTH,NJ
...,...,...,...,...,...,...,...,...,...
4654807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4654808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4654809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4654810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [263]:
columns_0 = data[years[0]].columns
year = years[3]
columns_1 = data[year].columns
print(f"Columns in {years[0]} but not in {year}")
print(set(columns_0) - set(columns_1))
print(f"Columns in {year} but not in {years[0]}")
print(set(columns_1) - set(columns_0))

Columns in 2010 but not in 2015
{'OTHER_WAGE_SOURCE_1', 'WAGE_RATE_FROM', 'WORK_LOCATION_STATE1', 'STATUS', 'NAICS_CODE', 'WORK_LOCATION_CITY1', 'SUBMIT', 'OTHER_WAGE_SOURCE_2', 'WAGE_RATE_TO', 'YR_SOURCE_PUB_1', 'YR_SOURCE_PUB_2', 'NUMBER'}
Columns in 2015 but not in 2010
{'H-1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'WORKSITE_COUNTY', 'WORKSITE_STATE', 'VISA_CLASS', 'EMPLOYER_PHONE', 'AGENT_ATTORNEY_STATE', 'AGENT_ATTORNEY_NAME', 'WAGE_UNIT_OF_PAY', 'EMPLOYER_PROVINCE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PHONE_EXT', 'WAGE_RATE_OF_PAY', 'NAIC_CODE', 'PREVAILING_WAGE', 'CASE_SUBMITTED', 'AGENT_ATTORNEY_CITY', 'CASE_STATUS', 'WORKSITE_CITY', 'CASE_NUMBER', 'WORKSITE_POSTAL_CODE', 'FULL_TIME_POSITION'}


In [260]:
for c in data[years[0]]:
    print(c)

NUMBER
STATUS
SUBMIT
DECISION_DATE
EMPLOYMENT_START_DATE
EMPLOYMENT_END_DATE
EMPLOYER_NAME
EMPLOYER_ADDRESS
EMPLOYER_CITY
EMPLOYER_STATE
EMPLOYER_POSTAL_CODE
SOC_CODE
SOC_NAME
JOB_TITLE
WAGE_RATE_FROM
WAGE_RATE_TO
TOTAL_WORKERS
WORK_LOCATION_CITY1
WORK_LOCATION_STATE1
OTHER_WAGE_SOURCE_1
YR_SOURCE_PUB_1
OTHER_WAGE_SOURCE_2
YR_SOURCE_PUB_2
NAICS_CODE
WAGE_OFFERED_UNIT_OF_PAY
